In [137]:
from tkinter import Tk
from tkinter.filedialog import askopenfilename
from tkinter.filedialog import asksaveasfilename
import pandas as pd
import openpyxl
from sys import platform
import numpy as np


In [157]:
Tk().withdraw()
filename = askopenfilename()
print(filename)


/Users/leecampbell/Dev/QMUL_SAD/final_apps_autumn.xlsx


In [158]:
print(platform)

darwin


In [170]:
df = pd.read_excel(filename,"dummy")


In [171]:
df['Total score'].fillna(df['Total score'].mean(), inplace=True)

In [172]:
group_df = df[['email', 'Course', 'YearOfStudy',
       'Gender', 'Bursary holder', 'Client meetings delivery', 'Total score',
       'Personality type']]

group_df = group_df.assign(groupno='')

In [173]:
group_df.sort_values(by=['Total score','YearOfStudy'], ascending=False, inplace=True)

In [180]:
def assign_group(df):
    count = 1
    up = True
    for i,row in group_df.iterrows():

        if count <= round(len(df)/6,0) and up == True:
            df.at[i,'groupno'] = count
            count += 1

        elif count > round(len(df)/6,0) and up == True:
            df.at[i,'groupno'] = count-1
            count-=1
            up = False

        if count > 1 and up == False:
            df.at[i,'groupno'] = count
            count -= 1
        elif count == 1 and up == False:
            df.at[i,'groupno'] = count
            up = True
    
    return df

new = assign_group(group_df)

    

In [175]:
new.groupby('groupno').mean()

,YearOfStudy,Total score
groupno,,
1,2.333333,7.000000
2,2.666667,7.083333
3,2.166667,7.250000
4,2.166667,7.416667
5,2.333333,7.333333
6,2.333333,7.250000
7,2.500000,7.250000
8,2.666667,7.083333
9,2.166667,7.028736


In [182]:
new.pivot_table(index=['groupno'], columns=['YearOfStudy'], aggfunc='size', fill_value=0)


YearOfStudy,1,2,3,4
groupno,,,,
1,0,4,2,0
2,0,3,2,1
3,0,5,1,0
4,1,4,0,1
5,1,2,3,0
6,0,4,2,0
7,0,3,3,0
8,1,1,3,1
9,1,3,2,0


In [141]:
new.pivot_table(index=['groupno'], columns=['Gender'], aggfunc='size', fill_value=0)


Gender,F,M
groupno,,
1,5,1
2,3,3
3,5,1
4,4,2
5,2,4
6,3,3
7,5,1
8,5,1
9,5,1


In [142]:
new['Gender'].value_counts()

F    41
M    19
Name: Gender, dtype: int64

In [179]:
book = openpyxl.load_workbook(filename)
writer = pd.ExcelWriter(filename, engine = 'openpyxl')
writer.book = book
new.to_excel(writer, sheet_name='New Groups')
writer.save()
writer.close()

'/Users/leecampbell/Dev/QMUL_SAD/final_apps_autumn.xlsx'